In [2]:
!pip install qiskit --quiet
!pip install qiskit_aer --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 82.8 MB/s eta 0:00:00


In [7]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [8]:
# ---------- ORACLES ----------
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)


def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced oracle: f(x) = x0 XOR x1 XOR ... XOR xn"""
    for q in inputs:
        qc.cx(q, ancilla)




In [9]:
# ---------- DEUTSCH–JOZSA CIRCUIT ----------
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    n: number of input qubits
    oracle_func: oracle function to modify the circuit
    oracle_args: extra arguments for oracle
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc


In [10]:
# ---------- EXECUTION ----------
def run_dj(qc):
    """Run Deutsch–Jozsa circuit on AerSimulator"""
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    if counts.get("0" * n, 0) == 1024:
        print("Function is CONSTANT")
    else:
        print("Function is BALANCED")


In [11]:
# ---------- MAIN ----------
if __name__ == "__main__":
    n = 3  # number of input qubits

    print("\n=== Constant Oracle (f(x)=0) ===")
    qc_const = deutsch_jozsa_circuit(
        n, oracle_constant, n, 0
    )
    print(qc_const.draw(fold=-1))
    run_dj(qc_const)

    print("\n=== Balanced Oracle (Parity) ===")
    qc_balanced = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n
    )
    print(qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)



=== Constant Oracle (f(x)=0) ===
     ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ H ├┤M├──────
     ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ H ├─╫─┤M├───
     ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├─╫──╫─┤M├
     ├───┤├───┤ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─╫──╫──╫─
     └───┘└───┘ ║  ║  ║ 
c: 3/═══════════╩══╩══╩═
                0  1  2 
Measurement counts: {'000': 1024}
Function is CONSTANT

=== Balanced Oracle (Parity) ===
     ┌───┐          ┌───┐     ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║       ║  ║ 
c: 3/══════════════════════════╩═══════╩══╩═
                               0       1  2 
Measurement counts: {'111': 1024}
Function is BALANCED


In [12]:
!pip install qiskit qiskit-aer qiskit-ibm-runtime --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.3 MB/s eta 0:00:00


In [13]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import Aer
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.quantum_info import Statevector
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

import numpy as np


Task 1: Modify the Oracle

In [ ]:
def nonlinear_balanced_oracle(qc, x, y, work):
    """
    Balanced function:
        f(x) = (x0 AND x1) XOR x2
    That is NOT parity.
    """
    # Compute x0 AND x1 into work qubit
    qc.ccx(x[0], x[1], work)
    # Apply (x0 AND x1) to target
    qc.cx(work, y)
    # Apply XOR with x2
    qc.cx(x[2], y)
    # Uncompute work qubit
    qc.ccx(x[0], x[1], work)

# Example demo with n=3 inputs:
x = QuantumRegister(3, 'x')
y = QuantumRegister(1, 'y')
w = QuantumRegister(1, 'w')
qc = QuantumCircuit(x, y, w)

qc.x(y) # Required for Deutsch-Jozsa
nonlinear_balanced_oracle(qc, x, y[0], w[0])
qc.draw()


x_0: ───────■──────────────■──
            │              │  
x_1: ───────■──────────────■──
            │              │  
x_2: ───────┼─────────■────┼──
     ┌───┐  │  ┌───┐┌─┴─┐  │  
  y: ┤ X ├──┼──┤ X ├┤ X ├──┼──
     └───┘┌─┴─┐└─┬─┘└───┘┌─┴─┐
  w: ─────┤ X ├──■───────┤ X ├
          └───┘          └───┘

Task 2: Change the Number of Input Qubits

In [ ]:
def deutsch_jozsa(n):
    x = QuantumRegister(n, 'x')
    y = QuantumRegister(1, 'y')
    w = QuantumRegister(1, 'w') if n >= 3 else None
    c = ClassicalRegister(n, 'c')

    qc = QuantumCircuit(x, y, *( [w] if w else [] ), c)

    # Initialize |y⟩ = |1⟩
    qc.x(y)

    # Apply H to all
    qc.h(x)
    qc.h(y)

    # Oracle
    if n >= 3:
        nonlinear_balanced_oracle(qc, x, y[0], w[0])
    else:
        qc.cx(x[0], y[0])  # fallback balanced function

    # Apply H to input register
    qc.h(x)

    # Measure input register
    qc.measure(x, c)

    return qc

for n in [2,4,5]:
    qc = deutsch_jozsa(n)
    result = Aer.get_backend("aer_simulator").run(transpile(qc)).result()
    counts = result.get_counts()
    print(f"\nn={n}, Circuit Depth: {qc.depth()}")
    print(counts)



n=2, Circuit Depth: 5
{'01': 1024}

n=4, Circuit Depth: 6
{'0111': 258, '0101': 256, '0110': 249, '0100': 261}

n=5, Circuit Depth: 6
{'00100': 271, '00111': 260, '00101': 249, '00110': 244}


Task 3:  Add Noise Simulation

In [ ]:
def noisy_sim(qc):
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01, 1), ["h", "x", "id"])
    noise_model.add_all_qubit_quantum_error(depolarizing_error(0.05, 2), ["cx"])

    backend = Aer.get_backend("aer_simulator")
    result = backend.run(transpile(qc, backend), noise_model=noise_model, shots=2048).result()
    return result.get_counts()

for n in [2,4,5]:
    qc = deutsch_jozsa(n)
    noisy_counts = noisy_sim(qc)
    print(f"\nNOISY results for n={n}:")
    print(noisy_counts)



NOISY results for n=2:
{'00': 72, '01': 1976}

NOISY results for n=4:
{'0001': 27, '0010': 29, '0000': 38, '0101': 459, '0100': 503, '0011': 17, '0111': 504, '0110': 471}

NOISY results for n=5:
{'00001': 25, '00000': 44, '00010': 20, '00110': 440, '00011': 25, '00111': 498, '00100': 525, '00101': 471}


Task 4: Run on IBM Quantum Device

In [14]:
!pip install --upgrade qiskit qiskit-aer qiskit-ibm-runtime --quiet


In [31]:
service = QiskitRuntimeService()
service.backends()


[<IBMBackend('ibm_fez')>,
 <IBMBackend('ibm_torino')>,
 <IBMBackend('ibm_marrakesh')>]

In [ ]:
qc = deutsch_jozsa(3)   # 3 input qubits DJ example
qc.draw('text')


┌───┐                         ┌───┐┌─┐      
x_0: ┤ H ├───────■──────────────■──┤ H ├┤M├──────
     ├───┤       │              │  ├───┤└╥┘┌─┐   
x_1: ┤ H ├───────■──────────────■──┤ H ├─╫─┤M├───
     ├───┤       │              │  ├───┤ ║ └╥┘┌─┐
x_2: ┤ H ├───────┼─────────■────┼──┤ H ├─╫──╫─┤M├
     ├───┤┌───┐  │  ┌───┐┌─┴─┐  │  └───┘ ║  ║ └╥┘
  y: ┤ X ├┤ H ├──┼──┤ X ├┤ X ├──┼────────╫──╫──╫─
     └───┘└───┘┌─┴─┐└─┬─┘└───┘┌─┴─┐      ║  ║  ║ 
  w: ──────────┤ X ├──■───────┤ X ├──────╫──╫──╫─
               └───┘          └───┘      ║  ║  ║ 
c: 3/════════════════════════════════════╩══╩══╩═
                                         0  1  2

In [55]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

def nonlinear_balanced_oracle(qc, x, y, work):
    qc.ccx(x[0], x[1], work)
    qc.cx(work, y)
    qc.cx(x[2], y)
    qc.ccx(x[0], x[1], work)

def deutsch_jozsa(n):
    x = QuantumRegister(n, 'x')
    y = QuantumRegister(1, 'y')
    w = QuantumRegister(1, 'w')
    c = ClassicalRegister(n, 'c')

    qc = QuantumCircuit(x, y, w, c)
    qc.x(y)
    qc.h(x)
    qc.h(y)
    nonlinear_balanced_oracle(qc, x, y[0], w[0])
    qc.h(x)
    qc.measure(x, c)
    return qc

qc = deutsch_jozsa(3)
qc.draw()


┌───┐                         ┌───┐┌─┐      
x_0: ┤ H ├───────■──────────────■──┤ H ├┤M├──────
     ├───┤       │              │  ├───┤└╥┘┌─┐   
x_1: ┤ H ├───────■──────────────■──┤ H ├─╫─┤M├───
     ├───┤       │              │  ├───┤ ║ └╥┘┌─┐
x_2: ┤ H ├───────┼─────────■────┼──┤ H ├─╫──╫─┤M├
     ├───┤┌───┐  │  ┌───┐┌─┴─┐  │  └───┘ ║  ║ └╥┘
  y: ┤ X ├┤ H ├──┼──┤ X ├┤ X ├──┼────────╫──╫──╫─
     └───┘└───┘┌─┴─┐└─┬─┘└───┘┌─┴─┐      ║  ║  ║ 
  w: ──────────┤ X ├──■───────┤ X ├──────╫──╫──╫─
               └───┘          └───┘      ║  ║  ║ 
c: 3/════════════════════════════════════╩══╩══╩═
                                         0  1  2

In [54]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import transpile

service = QiskitRuntimeService()

# Pick backend that supports standard measurement execution
backend = service.backend("ibm_torino")   # ✅ reliable static backend
print("Using backend:", backend.name)


Using backend: ibm_torino


In [ ]:
# Tried to run on the real backend IBM QCs, but facing multiple errors due of version mismatches and other things.

Task 5: Circuit Analysis

In [ ]:
x = QuantumRegister(3, 'x')
y = QuantumRegister(1, 'y')
w = QuantumRegister(1, 'w')
oracle = QuantumCircuit(x, y, w, name="NonlinearOracle")

nonlinear_balanced_oracle(oracle, x, y[0], w[0])

print("Oracle Gate Definition:\n")
print(oracle.to_gate().definition)

print("\nExplanation:")
print("""
f(x) = (x0 AND x1) XOR x2 is balanced because:
- x0 AND x1 = 1 only when x0=x1=1
- XOR with x2 flips exactly half of total states
→ Therefore, exactly 50% of inputs output 1 → BALANCED.
""")


Oracle Gate Definition:

                         
q_0: ──■──────────────■──
       │              │  
q_1: ──■──────────────■──
       │              │  
q_2: ──┼─────────■────┼──
       │  ┌───┐┌─┴─┐  │  
q_3: ──┼──┤ X ├┤ X ├──┼──
     ┌─┴─┐└─┬─┘└───┘┌─┴─┐
q_4: ┤ X ├──■───────┤ X ├
     └───┘          └───┘

Explanation:

f(x) = (x0 AND x1) XOR x2 is balanced because:
- x0 AND x1 = 1 only when x0=x1=1
- XOR with x2 flips exactly half of total states
→ Therefore, exactly 50% of inputs output 1 → BALANCED.

